In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime

import pandas as pd

from rasterio import features

from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

import rasterstats
import fiona

from tqdm import tqdm

from shapely.geometry import shape
from rasterstats.io import read_features

#from dask.diagnostics import ProgressBar

pbar = ProgressBar()
pbar.register()

pbar.unregister()

In [22]:
data = xr.open_mfdataset(r'C:\MAIACData\nc_monthly_daily\*.nc')['data']

In [23]:
# Subsetting params
x_start = 950
x_end = None

y_start = 950
y_stop = None

In [24]:
subset = data.isel(x=slice(x_start, y_stop), y=slice(y_start, y_stop))

In [25]:
subset = subset.isel(time=np.argsort(subset.time))

In [26]:
After2009 = subset.sel(time=slice('2009', '2016'))

In [27]:
monthly_data = After2009.resample('M', dim='time', how='mean', keep_attrs=True)

In [28]:
monthly_data

<xarray.DataArray 'data' (time: 66, y: 212, x: 290)>
dask.array<transpo..., shape=(66, 212, 290), dtype=float32, chunksize=(1, 212, 290)>
Coordinates:
  * x        (x) float64 2.461e+05 2.473e+05 2.486e+05 2.498e+05 2.511e+05 ...
  * y        (y) float64 2.356e+05 2.343e+05 2.33e+05 2.318e+05 2.305e+05 ...
  * time     (time) datetime64[ns] 2009-01-31 2009-02-28 2009-03-31 ...
Attributes:
    affine: [ -9.47639631e+05   1.25654304e+03   0.00000000e+00   1.42927781e+06
   0.00000000e+00  -1.25654304e+03]
    crs: +init=epsg:27700

In [29]:
data = monthly_data

The next few cells are the bits that convert the affine transform

In [30]:
# Get the actual Affine object from the data stored in the attrs
orig_aff = rasterio.Affine.from_gdal(*data.attrs['affine'])

In [31]:
def window_bounds(window, affine):
    (row_start, row_stop), (col_start, col_stop) = window
    w, s = (col_start, row_stop) * affine
    e, n = (col_stop, row_start) * affine
    return w, s, e, n

In [32]:
c, _, _, f = window_bounds( ( (x_start, 5000), (y_start, 5000)), orig_aff)  # c ~ west, f ~ north
a, b, _, d, e, _, _, _, _ = tuple(orig_aff)
new_aff = rasterio.Affine(a, b, c, d, e, f)

In [33]:
orig_aff

Affine(1256.5430440955893, 0.0, -947639.63051064778,
       0.0, -1256.5430440955893, 1429277.8120091767)

In [34]:
new_aff

Affine(1256.5430440955893, 0.0, 246076.26138016209,
       0.0, -1256.5430440955893, 235561.92011836683)

In [35]:
# Image to rasterize the polygons in to
rasterized_image = np.zeros(data.isel(time=0).shape, dtype=np.int)

# List to store dataframes in
dfs = []

feats = read_features(r'D:\Annies_Dissertation\Data\Boundaries\LSOA_Wessex.shp')

out_shape = data.isel(time=0).shape

This single line is the key thing that makes the code faster!

In [36]:
data = data.load()

KeyboardInterrupt: 

In [19]:
data

<xarray.DataArray 'data' (time: 66, y: 212, x: 290)>
array([[[  7.46854973,   7.31030512,   7.47690392, ...,          nan,
                  nan,          nan],
        [  6.85919046,   6.87198734,   5.62207413, ...,          nan,
                  nan,          nan],
        [  8.83175087,   9.8598671 ,   5.67711639, ...,          nan,
                  nan,          nan],
        ..., 
        [  8.56533813,   8.49901867,          nan, ...,          nan,
                  nan,          nan],
        [  8.56972504,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan]],

       [[ 15.02777386,   8.10723877,  12.05983162, ...,          nan,
                  nan,          nan],
        [         nan,          nan,   8.38002014, ...,          nan,
                  nan,          nan],
        [  9.77899742,  13.60194397,   9.22825813, ...,          na

In [20]:
# Loop over features (polygons) in the shapefile
for f in tqdm(feats):
    # Rasterize the polygon into an array
    rasterized_image = features.rasterize([(shape(f['geometry']),1)],
                                          out_shape=out_shape,
                                          transform=new_aff,
                                          fill=0,
                                          all_touched=True)

    # Extract from the xarray where the rasterized polygon is
    region = data.where(rasterized_image == 1)
    
    # Combine x and y into a new dimension called allpoints and calculate the mean over it
    # and then convert to a dataframe with an appropriate name
    res = region.stack(allpoints=['x','y']).mean(dim='allpoints').to_dataframe(name=f['properties']['LSOA11CD'])
    
    # Append to the list of data frames so we can concatenate them all at the end
    dfs.append(res)
    
stats = pd.concat(dfs, axis=1)

2575it [01:09, 36.38it/s]


In [37]:
stats

E01014869  E01014890  E01015272  E01015273  E01015274  E01015275  \
time                                                                           
2009-01-31   4.894197   6.379436  11.609479  11.410336  11.650494  11.682238   
2009-02-28   8.269163   8.117812  16.521988  17.099459  17.253952  19.085062   
2009-03-31  16.443802  15.787933  24.719713  23.110231  22.489740  24.195459   
2009-04-30  12.831035  12.642556  13.966952  15.072072  14.637295  12.367358   
2009-05-31   9.983700   6.370895  14.851036  14.757053  14.214451  13.850450   
2009-06-30  14.679679  13.817136  15.499490  17.341637  18.810036  15.580470   
2009-07-31  11.896524  12.728112  21.609333  22.213881  22.655102  21.706160   
2009-08-31  17.366266   8.997812  14.671386  15.520734  15.764976  14.220521   
2009-09-30  11.546007   9.072837  16.571581  15.893867  15.805025  16.007593   
2009-10-31   8.697833   9.306887  14.976901  15.424840  15.962790  16.716471   
2009-11-30   5.488342   5.816244  13.472143  13.326997  14.088587  13.691173   
2009-12-31   9.032913   9.865499   8.869758   9.490275   9.318064   8.659707   
2010-01-31   8.074108   7.211493   7.773952   9.367728   9.678822   7.477477   
2010-02-28  14.925032  15.940673  19.779760  18.430952  18.193981  19.717033   
2010-03-31   9.648796   7.933636  13.292311  12.227306  10.803463  10.926511   
2010-04-30  12.068913  10.465383  18.364841  18.523458  17.657255  17.764023   
2010-05-31   9.601681   8.533235  11.480257  12.787391  13.428074  10.813763   
2010-06-30   9.306105   8.149064  10.728844  12.440989  13.721837  11.713570   
2010-07-31   8.165253  11.479817  17.084770  16.915024  17.047279  17.542801   
2010-08-31   6.208467   7.386232  11.730207  12.080920  13.210400  13.161413   
2010-09-30  10.867139   7.016661  15.078695  15.064517  15.612456  15.228891   
2010-10-31   4.577707   4.467451  10.061832  10.282275  10.692778  10.815086   
2010-11-30   6.014006   4.241180   7.897810   8.435908   9.507037   8.463559   
2010-12-31   8.668041  10.178349  13.143812  10.284458   9.310347  14.530696   
2011-01-31   5.582096   4.423071  12.289507  11.958020  12.871197  14.068691   
2011-02-28   5.270328   4.596809  13.688326  13.819097  16.951843  20.381651   
2011-03-31  11.506182  14.474870  34.112225  32.733517  32.616272  35.271183   
2011-04-30  14.254741  15.914761  18.110867  18.279181  19.165329  19.201267   
2011-05-31  14.729631  12.670700  13.034283  14.121380  16.152287  13.561242   
2011-06-30   9.034540   9.912970  10.614342  10.543361   9.718155   9.274674   
...               ...        ...        ...        ...        ...        ...   
2012-01-31   9.754252   9.596786  11.191071  11.337541  11.944909  12.153346   
2012-02-29   7.640415   8.239361  14.836639  14.610316  15.031425  15.970848   
2012-03-31  13.653416  11.842598  21.788055  22.170935  22.640127  21.882195   
2012-04-30  13.737553  11.628886  16.828392  16.671410  15.748659  16.006239   
2012-05-31  11.686541   9.040792  13.291565  13.463837  14.140222  13.521429   
2012-06-30  24.576422  21.900684  12.297523  13.175916  14.482362  14.633374   
2012-07-31   7.426673   6.503408   9.617847  10.371747  10.734545  10.044309   
2012-08-31  20.222605  16.759089  15.975445  16.967167  17.992277  16.784550   
2012-09-30   8.662121   7.829556   9.606607  10.744453  10.957172   8.618340   
2012-10-31   5.789569   8.922488   9.621822  10.237657  11.583183  11.694120   
2012-11-30   4.140850   2.914655   8.584433   9.340146  11.760993   9.258857   
2012-12-31   4.527712   5.259722   6.963664   7.789634   7.921900   6.655417   
2013-01-31   6.698851   5.239190   8.566266   8.649814   8.358732   8.949495   
2013-02-28  10.370106  12.113716  15.102678  14.234664  14.303762  17.645443   
2013-03-31   5.248752  10.153851  26.855774  25.281967  25.976355  26.564987   
2013-04-30   8.094871   7.991707   9.966362  10.334037  10.321016  11.705468   
2013-05-31   5.898426   8.187919  12.024889  12.036827  12.623138  12.552957   
2013-06-30  

In [38]:
stats = stats.dropna(how='all')

In [39]:
melted_stats = pd.melt(stats.reset_index(), id_vars='time', var_name='LSOA').dropna()

In [40]:
melted_stats

time       LSOA      value
0      2009-01-31  E01014869   4.894197
1      2009-02-28  E01014869   8.269163
2      2009-03-31  E01014869  16.443802
3      2009-04-30  E01014869  12.831035
4      2009-05-31  E01014869   9.983700
5      2009-06-30  E01014869  14.679679
6      2009-07-31  E01014869  11.896524
7      2009-08-31  E01014869  17.366266
8      2009-09-30  E01014869  11.546007
9      2009-10-31  E01014869   8.697833
10     2009-11-30  E01014869   5.488342
11     2009-12-31  E01014869   9.032913
12     2010-01-31  E01014869   8.074108
13     2010-02-28  E01014869  14.925032
14     2010-03-31  E01014869   9.648796
15     2010-04-30  E01014869  12.068913
16     2010-05-31  E01014869   9.601681
17     2010-06-30  E01014869   9.306105
18     2010-07-31  E01014869   8.165253
19     2010-08-31  E01014869   6.208467
20     2010-09-30  E01014869  10.867139
21     2010-10-31  E01014869   4.577707
22     2010-11-30  E01014869   6.014006
23     2010-12-31  E01014869   8.668041
24     2011-01-31  E01014869   5.582096
25     2011-02-28  E01014869   5.270328
26     2011-03-31  E01014869  11.506182
27     2011-04-30  E01014869  14.254741
28     2011-05-31  E01014869  14.729631
29     2011-06-30  E01014869   9.034540
...           ...        ...        ...
170052 2012-01-31  E01033384   8.309803
170053 2012-02-29  E01033384  14.266115
170054 2012-03-31  E01033384  15.893249
170055 2012-04-30  E01033384  11.736242
170056 2012-05-31  E01033384  10.997645
170057 2012-06-30  E01033384  23.041363
170058 2012-07-31  E01033384   9.831959
170059 2012-08-31  E01033384  13.528056
170060 2012-09-30  E01033384  11.343671
170061 2012-10-31  E01033384   9.390759
170062 2012-11-30  E01033384   6.440351
170063 2012-12-31  E01033384   6.026381
170064 2013-01-31  E01033384   6.992032
170065 2013-02-28  E01033384   9.205453
170066 2013-03-31  E01033384  12.879792
170067 2013-04-30  E01033384   4.648635
170068 2013-05-31  E01033384   8.394756
170069 2013-06-30  E01033384  10.300530
170070 2013-07-31  E01033384  14.623461
170071 2013-08-31  E01033384  12.760908
170072 2013-09-30  E01033384  11.320148
170073 2013-10-31  E01033384  11.374761
170074 2013-11-30  E01033384   7.701131
170075 2013-12-31  E01033384   7.387822
170076 2014-01-31  E01033384   6.378034
170077 2014-02-28  E01033384  12.878300
170078 2014-03-31  E01033384  22.636251
170079 2014-04-30  E01033384  17.177961
170080 2014-05-31  E01033384  13.751208
170081 2014-06-30  E01033384  15.409837

[169981 rows x 3 columns]

In [41]:
melted_stats['month'] = melted_stats.time.dt.month

In [42]:
melted_stats['year'] = melted_stats.time.dt.year

In [43]:
melted_stats.head()

time       LSOA      value  month  year
0 2009-01-31  E01014869   4.894197      1  2009
1 2009-02-28  E01014869   8.269163      2  2009
2 2009-03-31  E01014869  16.443802      3  2009
3 2009-04-30  E01014869  12.831035      4  2009
4 2009-05-31  E01014869   9.983700      5  2009

In [44]:
melted_stats.to_csv(r'D:\Annies_Dissertation\Analysis\Regression\Monthly_PM25_LSOA.csv')